In [1]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
from itertools import compress
import nltk
import en_core_web_lg
nlp_spacy = en_core_web_lg.load()

In [47]:
data = pd.read_csv('output_combined.tsv',delimiter='\t')

In [48]:
data_1 = pd.read_csv('article_data_V2.csv')

In [49]:
data_1 = data_1.truncate(after=1087)
data_1.shape

(1088, 7)

In [51]:
snippets_list = []
for idx, row in tqdm(data.iterrows()):
        temp_dict = dict()
        temp_dict['title'] = row['title']
        temp_dict['url'] = row['url']
        temp_dict['body'] = row['body']
        temp_dict['company_name'] = row['company_name']
        snippets_list.append(temp_dict)
for idx, row in tqdm(data_1.iterrows()):
        temp_dict = dict()
        temp_dict['title'] = row['title']
        temp_dict['url'] = row['url']
        temp_dict['body'] = row['data']
        temp_dict['company_name'] = row['company_name']
        snippets_list.append(temp_dict)

data_new = pd.DataFrame(snippets_list)
    

505it [00:00, 21396.05it/s]
1088it [00:00, 29708.11it/s]


In [52]:
data_new.shape, data_new.columns

((1593, 4), Index(['title', 'url', 'body', 'company_name'], dtype='object'))

In [53]:
data_new.iloc[-1]

title           Intellectual Property Rights During -- and Aft...
url             https:\/\/www.mddionline.com\/legal\/intellect...
body            Manufacturers came to the rescue to quickly de...
company_name                                   ford_motor_company
Name: 1592, dtype: object

In [54]:
data_new.to_csv("data_bef_prep.csv")

In [2]:
data_aft_prep = pd.read_csv('data.csv')

In [3]:
data_aft_prep.columns

Index(['Unnamed: 0', 'text', 'article_id', 'company_name', 'title', 'id'], dtype='object')

In [3]:
comp_list = ['alphabet','google','facebook','ITV','Pearson','BT group','BT','Vodafone group','vodafone','burberry group','burberry','nike','nike inc','compass group',
'restaurant group','trg','amazon','amazon in','booking holdings','kingfisher','marks and spencer group','marks and spencer','marks & spencer group','marks & spencer',
'next plc','tjx','tjx companies','sainsbury','tesco','british american tobacco','coca-cola','coca cola','diageo','imperial brands','pepsico',
'PHILIP MORRIS INTERNATIONAL','philip morris','TATE AND LYLE','TATE & LYLE','PROCTER & GAMBLE','PROCTER and GAMBLE','p&g','RECKITT BENCKISER GROUP','RECKITT BENCKISER',
'RECKITT','RB','unilever','BAKER HUGHES','BP','CABOT OIL & GAS CORP','coterra energy','CHEVRON CORP','CHEVRON','EOG RESOURCES','eog','EXXON MOBIL','EXXONMOBIL',
'HALLIBURTON','PHILLIPS','PIONEER NATURAL RESOURCE','PIONEER NATURAL RESOURCES','VALERO ENERGY','VALERO','BANK OF AMERICA','BARCLAYS','CITIGROUP','CITI',
'FIRST REPUBLIC BANK','FIRST REPUBLIC','JPMORGAN CHASE','JPMORGAN','JP MORGAN','J.P. MORGAN','LLOYDS BANKING GROUP','LLOYDS BANK','NATWEST GROUP','NATWEST',
'STANDARD CHARTERED','STANDARD CHARTERED PLC','WELLS FARGO','3I GROUP','3I','AMERICAN EXPRESS','BERKSHIRE HATHAWAY INC','BERKSHIRE HATHAWAY','BERKSHIRE',
'HATHAWAY','CME GROUP','HARGREAVES LANSDOWN','LONDON STOCK EXCHANGE GROUP','LSEG','LONDON STOCK EXCHANGE','AVIVA','DIRECT LINE INSURANCE','DIRECT LINE group',
'DIRECT LINE','LEGAL AND GENERAL GROUP','LEGAL AND GENERAL','legal & general','PHOENIX GROUP','phoenix','PROGRESSIVE CORP','PROGRESSIVE CORPORATION','PROGRESSIVE','PRUDENTIAL','ANTHEM',
'BOSTON SCIENTIFIC','CONVATEC GROUP','CONVATEC','DANAHER CORPORATION','DANAHER CORP','DANAHER','SMITH AND NEPHEW','SMITH & NEPHEW','ASTRAZENECA','GLAXOSMITHKLINE',
'GSK','ILLUMINA','JOHNSON & JOHNSON',"JOHNSON's",'J&J','THERMO FISHER SCIENTIFIC','THERMO FISHER','ASHTEAD GROUP','ASHTEAD','BAE SYSTEMS','BUNZL','ELECTROCOMPONENTS',
'FERGUSON PLC','FERGUSON','IMI PLC','IMI','MELROSE INDUSTRIES','MELROSE','QINETIQ GROUP','QINETIQ','RAYTHEON TECHNOLOGIES','RAYTHEON','SMITHS GROUP','SMITHS',
'HAYS PLC','HAYS','IHS MARKIT','RELX group','RELX','RENTOKIL INITIAL','RENTOKIL','SERCO GROUP','SERCO','FIRSTGROUP','GO-AHEAD GROUP','GO-AHEAD','ROYAL MAIL group',
'ROYAL MAIL','ADVANCED MICRO DEVICES','AMD','QORVO','TEXAS INSTRUMENT','TEXAS INSTRUMENTS','ADOBE','ATLASSIAN','ATLASSIAN CORPORATION','AVEVA GROUP','AVEVA',
'CAPITA GROUP','CAPITA','CTS','Cognizant','FDM GROUP HOLDINGS','FDM GROUP','FDM','FIDELITY NATIONAL INFORMATION SERVICES','FNI Services','GODADDY','INTUIT',
'MASTERCARD','MICROSOFT CORP','MICROSOFT CORPORATION','MICROSOFT','OKTA','PAYCHEX','PAYPAL','SNOWFLAKE','SS AND C TECHNOLOGIES','SS&C TECHNOLOGIES','SS&C',
'VISA INC','VISA','ZOOM VIDEO COMMUNICATIONS','ZOOM VIDEO','ZOOM COMMUNICATIONS','AMPHENOL CORP','AMPHENOL CORPORATION','AMPHENOL','CDW','CISCO SYSTEMS','CISCO',
'HP','Hewlett-Packard','MOTOROLA SOLUTIONS','MOTOROLA','QUALCOMM','SPIRENT COMMUNICATIONS','SPIRENT','ANGLO AMERICAN','BHP GROUP','BHP','CRODA INTERNATIONAL','CRODA',
'ESSENTRA','INTERNATIONAL FLAVORS & FRAGRANCES','IFF','JOHNSON MATTHEY','JMAT','MARTIN MARIETTA MATERIALS INC','MARTIN MARIETTA MATERIALS','MONDI','MONDI GROUP',
'NEWMONT','NEWMONT CORPORATION','SMITH (DS)','DS Smith','ASSURA REIT','ASSURA FINANCING','ASSURA','CAPITAL & COUNTIES PROPERTIES','CAPITAL & COUNTIES','CROWN CASTLE',
'CROWN CASTLE INTERNATIONAL CORP','CROWN CASTLE INTERNATIONAL CORPORATION','EQUINIX REIT','EQUINIX','EQUITY LIFESTYLE PROPERTIES','ELS','LAND SECURITIES GROUP',
'LANDSEC','LONDONMETRIC PROPERTY','LMP','SBA COMMUNICATIONS','SBA COMMUNICATIONS CORPORATION','SBAC','SEGRO PLC','SEGRO','SHAFTESBURY PLC','SHAFTESBURY',
'SUN COMMUNITIES','AMERICAN ELECTRIC POWER','american electric','AEP','CENTRICA','EXELON CORPORATION','EXELON','NATIONAL GRID','NEXTERA ENERGY','NEXTERA','PUBLIC SERVICE ENTERPRISE GROUP',
'PSEG','UNITED UTILITIES GROUP','UNITED UTILITIES','WEC ENERGY GROUP','WEC ENERGY','SSE PLC','GENERAL MOTORS','LEAR CORPORATION','LEAR','LEAR CORP','STARBUCKS',
'EBAY','TESLA INC','TESLA',"MCDONALD'S CORP","MCDONALD'S",'WALMART','WALMART CORPORATE','COLGATE-PALMOLIVE','COLGATE','3M','3M SCIENCE','ROLLS-ROYCE HOLDINGS','ROLLS-ROYCE',
'EQUIFAX','FEDEX CORP','FEDEX CORPORATION','FEDEX','UBER TECHNOLOGIES','UBER','GENERAL ELECTRIC','GE','LOCKHEED MARTIN CORPORATION','LOCKHEED MARTIN','NVIDIA CORPORATION',
'NVIDIA','APPLE INC','APPLE','XCEL ENERGY','AUTOHOME','BAIDU','KAKAKU','NETEASE','NINTENDO','NINTENDO LTD','NIPPON TELEVISION','NIPPON TV','TENCENT HOLDINGS',
'TENCENT','BHARTI AIRTEL','AIRTEL','KDDI CORP','KDDI CORPORATION','KDDI','NIPPON TELEGRAPH AND TELEPHONE CORPORATION','NTT CORP','SOFTBANK CORP','SOFTBANK GROUP',
'SOFTBANK','AISIN CORP','AISIN','BRIDGESTONE CORPORATION','BRIDGESTONE GROUP','BRIDGESTONE','DONGFENG MOTOR GROUP','HONDA MOTOR COMPANY','HONDA','ISUZU MOTORS LTD',
'ISUZU MOTORS','ISUZU','MAHINDRA AND MAHINDRA LTD','MAHINDRA & MAHINDRA LTD','MAHINDRA & MAHINDRA','Mahindra GROUP','SUBARU CORP','SUBARU CORPORATION','SUBARU',
'SUMITOMO ELECTRIC INDUSTRIES','SUMITOMO ELECTRIC','SUZUKI MOTOR CORP','SUZUKI MOTOR CORPORATION','SUZUKI MOTOR','SUZUKI','TATA MOTORS LTD','TATA MOTORS',
'TOYOTA MOTOR CORP','TOYOTA MOTOR CORPORATION','TOYOTA','XPENG INC','XPENG INC.','XPENG','YAMAHA MOTOR COMPANY','YAMAHA MOTORS','YAMAHA','YAMAHA MOTOR CO., LTD.',
'YAMAHA MOTOR','PARKER-HANNIFIN CORP','PARKER-HANNIFIN','PARKER HANNIFIN','Nestlé India','Nestlé','Nestle','PFIZER INC','PFIZER','msci','msci inc','salesforce','itochu',
'komatsu','airbnb','sage','expedia','bajaj finance','twilio','whitbread','otis','japan tobacco','daifuku','ACCIONA','ALDI','AMP Capital','ASDA','AXA','Actelion','Air Liquide','Aramco','Aurora Cannabis','Aviva','AVIVA PLC','Bank Hapoalim BM','BestBuy','Biffa','Big Society Capital','Blackrock','Boeing','British Airways','Cambridge Glasshouse Company','Castrol','credit Suisse','DWS','Deutsche Bank','Dewan Housing Finance Corporation Ltd','Discovery Limited','Dove','Drax','E.ON','ESSECO','Eastman Kodak','Enel','Eni','FMC','Flipkart','Ford', 'Ford Motor Company','Freedom Foods Group Ltd', 'GM+Shell','Generali','Goldman Sachs','Granville','H&M','HAZAMA ANDO CORP','HS2','Hanson','Heathrow','Heineken','ICICI Bank','ING','Infosys','Jhonson Controls','John Lewis','KLM Airlines','KPMG','Kao data','Kobe Steel','Konkola Copper Mines','LG Polymers','LeasePlan','Lego','Lombard Odier','Luckin','Luckin Coffee Inc', 'Mars','Mercedes Benz','Metlife','Monzo Bank', 'Morrisons','Anglo American','Salzgitter','Arrival','First Bus','EIB','Enel','Flynn','Primark','Shell', 'Rolls-Royce', 'Tevva', 'Vattenfall','Violia','Carbon Clean','Nishimatsu Construction Co., Ltd.', 'Nokia','O2', 'OVO Energy','Polestar','SQUAD','Stena Bulk', 'Takeda', 'Target','Toshiba IT-Services Corp', 'Total','Turing Pharma','Valeant Pharma','Yes Bank Limited','Zhenhua','abu_dhabi_national_oil_company','agricultural bank of china','alibaba', 'Alibaba Group Holding Limited', 'Alibaba Group Holding', 'Alibaba.com','alibaba_group',]


comp_list = [each_string.lower() for each_string in comp_list]


In [4]:
def ent_org(text):
        lst=[]
        doc = nlp_spacy(text)
        for word in doc.ents:
            if(word.label_=='ORG'):
                low_txt = word.text.lower()
                lst.append(low_txt)
        return lst


In [6]:
def one_comp_title(snippet_company,title_company):
    snippet_company = snippet_company[:-1]
    comp_lst=snippet_company.split(",")
    
    if(all(elem == comp_lst[0] for elem in comp_lst) and comp_lst[0]==title_company):     #all the companies found in snippet level are belonging to company found at article
        return '4-1',title_company
    
    elif(any(map(lambda v: v in title_company.split(","), comp_lst))):              #any one of the companies found in snippet level are belonging to company found at article
        return '4-2',title_company
    
    else:                                                                           #company found at snippet level are not belonging to company found at article
        return '4-3','None'

def mul_comp_title(snippet_company,title_company):
    snippet_company = snippet_company[:-1]
    comp_lst=snippet_company.split(",")
    
    if(all(elem == comp_lst[0] for elem in comp_lst) and any(map(lambda v: v in title_company.split(","), comp_lst))):      #all the companies found in snippet level are belonging to any one of the company found at article
        return '7-1',snippet_company.split(",")[0]
    
    elif(any(map(lambda v: v in title_company.split(","), comp_lst))):                                                      #any one of the companies in snippet level are belonging to any one of the company found at article
        lst = list(map(lambda v: v in title_company.split(","), comp_lst))
        return '7-2',','. join(set(compress(comp_lst, lst)))
    
    else:                                                                                                                   #company found at snippet level are not belonging to any company found at article
        return '7-3','None'


def tag(snippet,title):
    k=0
    #title = title
    #snippet = data_list[id-(key*15)+1]
    title_company=""
    snippet_company=""

    lst_art = ent_org(title) #get ORG tagged texts for Article title
    lst_snip = ent_org(snippet) #get ORG tagged texts for snippets

    for company in comp_list:          #Check for ORG texts in master list
        if(company.lower() in lst_art):
            k=1
            title_company=title_company+company+","
        if(company.lower() in lst_snip):
            snippet_company=snippet_company+company+","

    #Code for the snippets, where article title do not have company name
    if(k==0):
        
        snippet_company = snippet_company[:-1]
        
        if not snippet_company:     #Getting coref data, since snippet don't have any company
            return '1',"coref","None"
        else:                                          #having company in snippet
            return '3',snippet_company,"None"
        
    #Code for the snippets, where article title have company name
    else:
        title_company = title_company[:-1]

        if(len(title_company.split(","))==1):       #Article title having single company
            
            if(snippet_company):                    #snippet having company name
                snippet_company = snippet_company[:-1]
                comp_lst=snippet_company.split(",")
                
                if(all(elem == comp_lst[0] for elem in comp_lst) and comp_lst[0]==title_company):     #all the companies found in snippet level are belonging to company found at article
                    return '4-1',title_company,title_company
                
                elif(any(map(lambda v: v in title_company.split(","), comp_lst))):              #any one of the companies found in snippet level are belonging to company found at article
                    return '4-2',title_company,title_company
                
                else:                                                                           #company found at snippet level are not belonging to company found at article
                    return '4-3','None',title_company
                #return one_comp_title(snippet_company,title_company),title_company

            else:
                return '6',title_company,title_company
        
        else:                                       #Article title having multiple companies
        
            if(snippet_company):                     #Snippet have company name
                snippet_company = snippet_company[:-1]
                comp_lst=snippet_company.split(",")
                
                if(all(elem == comp_lst[0] for elem in comp_lst) and any(map(lambda v: v in title_company.split(","), comp_lst))):      #all the companies found in snippet level are belonging to any one of the company found at article
                    return '7-1',snippet_company.split(",")[0],title_company
                
                elif(any(map(lambda v: v in title_company.split(","), comp_lst))):                                                      #any one of the companies in snippet level are belonging to any one of the company found at article
                    lst = list(map(lambda v: v in title_company.split(","), comp_lst))
                    return '7-2',','. join(set(compress(comp_lst, lst))),title_company
                
                else:                                                                                                                   #company found at snippet level are not belonging to any company found at article
                    return '7-3','None',title_company
                #return mul_comp_title(snippet_company,title_company),title_company

            else:
                return '9',title_company,title_company


In [8]:
len(data_aft_prep)

64138

In [16]:
no_tit_no_snip=0
no_tit=0
no_tit_snip=0
sin_tit_no_snip=0
yes_tit=0
sin_tit=0
mul_tit_no_snip=0
mul_tit=0
sin_tit_all_snip=0
sin_tit_any_snip=0
sin_tit_oth_snip=0
mul_tit_all_snip=0
mul_tit_any_snip=0
mul_tit_oth_snip=0
final_snippend_list=[]

for idx, row in tqdm(data_aft_prep.iterrows()):
    if(str(row['text'])!='nan' and str(row['title'])!='nan'):
        value,snippet_org, title_org = tag(row['text'],row['title'])
    else:
        value='nan'
        snippet_org='nan'
        title_org='nan'
    temp_dict=dict()
    temp_dict['article_id'] = row['article_id']
    temp_dict['title'] = row['title']
    temp_dict['snippet'] = row['text']
    temp_dict['company_name'] = row['company_name']
    #temp_dict['snippet_coref'] = coref
    temp_dict['snippet_org'] = snippet_org
    #temp_dict['snippet_coref_org'] = snippet_coref_org
    temp_dict['title_org'] = title_org
    if(snippet_org == title_org):
        temp_dict['tit_sni_same'] = 1
    else:
        temp_dict['tit_sni_same'] = 0
    #temp_dict['coref_tit_sni_same'] = bool2
    temp_dict['id'] = row['id']
    temp_dict['value'] = value
    if(value == '1'):
        no_tit_no_snip+=1
        no_tit+=1
    elif(value == '3'):
        no_tit_snip+=1
        no_tit+=1
    elif(value == '6'):
        sin_tit_no_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '9'):
        mul_tit_no_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '4-1'):
        sin_tit_all_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '4-2'):
        sin_tit_any_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '4-3'):
        sin_tit_oth_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '7-1'):
        mul_tit_all_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '7-2'):
        mul_tit_any_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '7-3'):
        mul_tit_oth_snip+=1
        yes_tit+=1
        mul_tit+=1
    
    final_snippend_list.append(temp_dict)
snippet_df = pd.DataFrame(final_snippend_list) 
    

64138it [10:58, 97.40it/s] 


In [19]:
snippet_df.to_csv("emperical_result.csv")

In [21]:
print("Total no of sentences : ",len(data_aft_prep))
print(yes_tit)
print()
print("Percentage of sentences with company in title : "+str(round((yes_tit/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit)
print()
print("Percentage of sentences with no company in title : "+str(round((no_tit/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit)
print()
print("Percentage of sentences with single company in title : "+str(round((sin_tit/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit)
print()
print("Percentage of sentences with multiple companies in title : "+str(round((mul_tit/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit_no_snip)
print()
print("Percentage of sentences with no company in title as well as snippet : "+str(round((no_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit_snip)
print()
print("Percentage of sentences with no company in title and company in snippet : "+str(round((no_tit_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_no_snip)
print()
print("Percentage of sentences with single company in title and no company in snippet : "+str(round((sin_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_all_snip)
print()
print("Percentage of sentences with single company in title and snippet have title company : "+str(round((sin_tit_all_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_any_snip)
print()
print("Percentage of sentences with single company in title and one of the companies in snippet is title company : "+str(round((sin_tit_any_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_oth_snip)
print()
print("Percentage of sentences with single company in title and snippet company is not matching title company : "+str(round((sin_tit_oth_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_no_snip)
print()
print("Percentage of sentences with multiple companies in title and no company in snippet : "+str(round((mul_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_all_snip)
print()
print("Percentage of sentences with multiple companies in title and snippet having one of the company in title : "+str(round((mul_tit_all_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_any_snip)
print()
print("Percentage of sentences with multiple companies in title and any one of the companies in snippet is same as one of the title company : "+str(round((mul_tit_any_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_oth_snip)
print()
print("Percentage of sentences with multiple companies in title and snippet company is matching none of the title company : "+str(round((mul_tit_oth_snip/len(data_aft_prep))*100,2))+"%")
print()

Total no of sentences :  64138
10700

Percentage of sentences with company in title : 16.68%

53400

Percentage of sentences with no company in title : 83.26%

9533

Percentage of sentences with single company in title : 14.86%

1167

Percentage of sentences with multiple companies in title : 1.82%

50561

Percentage of sentences with no company in title as well as snippet : 78.83%

2839

Percentage of sentences with no company in title and company in snippet : 4.43%

7173

Percentage of sentences with single company in title and no company in snippet : 11.18%

2071

Percentage of sentences with single company in title and snippet have title company : 3.23%

83

Percentage of sentences with single company in title and one of the companies in snippet is title company : 0.13%

206

Percentage of sentences with single company in title and snippet company is not matching title company : 0.32%

973

Percentage of sentences with multiple companies in title and no company in snippet : 1.52%



In [2]:
result = pd.read_csv('emperical_result.csv')

In [6]:
lst = result[result.value=='3'].snippet_org# or result.value=='4-3' or result.value=='3']

In [11]:
lst = list(set(lst))
count=0
for i in lst:
    lst_spl = i.split(",")
    if(len(lst_spl)>1):
        count+=1
print(count)

158


In [5]:
def one_comp_title(snippet_company,title_company):
    snippet_company = snippet_company[:-1]
    comp_lst=snippet_company.split(",")
    
    if(all(elem == comp_lst[0] for elem in comp_lst) and comp_lst[0]==title_company):     #all the companies found in snippet level are belonging to company found at article
        return '4-1',title_company
    
    elif(any(map(lambda v: v in title_company.split(","), comp_lst))):              #any one of the companies found in snippet level are belonging to company found at article
        return '4-2',title_company
    
    else:                                                                           #company found at snippet level are not belonging to company found at article
        return '4-3','None'

def mul_comp_title(snippet_company,title_company):
    snippet_company = snippet_company[:-1]
    comp_lst=snippet_company.split(",")
    
    if(all(elem == comp_lst[0] for elem in comp_lst) and any(map(lambda v: v in title_company.split(","), comp_lst))):      #all the companies found in snippet level are belonging to any one of the company found at article
        return '7-1',snippet_company.split(",")[0]
    
    elif(any(map(lambda v: v in title_company.split(","), comp_lst))):                                                      #any one of the companies in snippet level are belonging to any one of the company found at article
        lst = list(map(lambda v: v in title_company.split(","), comp_lst))
        return '7-2',','. join(set(compress(comp_lst, lst)))
    
    else:                                                                                                                   #company found at snippet level are not belonging to any company found at article
        return '7-3','None'


def tag(snippet,title,coref_flag):
    k=0
    #title = title
    #snippet = data_list[id-(key*15)+1]
    title_company=""
    snippet_company=""

    lst_art = ent_org(title) #get ORG tagged texts for Article title
    lst_snip = ent_org(snippet) #get ORG tagged texts for snippets

    for company in comp_list:          #Check for ORG texts in master list
        if(company.lower() in lst_art):
            k=1
            title_company=title_company+company+","
        if(company.lower() in lst_snip):
            snippet_company=snippet_company+company+","

    #Code for the snippets, where article title do not have company name
    if(k==0):
        
        snippet_company = snippet_company[:-1]
        
        if not snippet_company and coref_flag==0:     #Getting coref data, since snippet don't have any company
            return '1',"coref","None"
        elif not snippet_company and coref_flag==1:
                return '2',"None","None"
        else:                                          #having company in snippet
            return '3',snippet_company,"None"
        
    #Code for the snippets, where article title have company name
    else:
        title_company = title_company[:-1]

        if(len(title_company.split(","))==1):       #Article title having single company
            
            if(snippet_company):                    #snippet having company name
                snippet_company = snippet_company[:-1]
                comp_lst=snippet_company.split(",")
                
                if(all(elem == comp_lst[0] for elem in comp_lst) and comp_lst[0]==title_company):     #all the companies found in snippet level are belonging to company found at article
                    return '4-1',title_company,title_company
                
                elif(any(map(lambda v: v in title_company.split(","), comp_lst))):              #any one of the companies found in snippet level are belonging to company found at article
                    return '4-2',title_company,title_company
                
                else:                                                                           #company found at snippet level are not belonging to company found at article
                    return '4-3','None',title_company
                #return one_comp_title(snippet_company,title_company),title_company
            elif coref_flag==0:                                   #Snippet don't have company name
                    return '5',"coref",title_company
            else:
                return '6',title_company,title_company
        
        else:                                       #Article title having multiple companies
        
            if(snippet_company):                     #Snippet have company name
                snippet_company = snippet_company[:-1]
                comp_lst=snippet_company.split(",")
                
                if(all(elem == comp_lst[0] for elem in comp_lst) and any(map(lambda v: v in title_company.split(","), comp_lst))):      #all the companies found in snippet level are belonging to any one of the company found at article
                    return '7-1',snippet_company.split(",")[0],title_company
                
                elif(any(map(lambda v: v in title_company.split(","), comp_lst))):                                                      #any one of the companies in snippet level are belonging to any one of the company found at article
                    lst = list(map(lambda v: v in title_company.split(","), comp_lst))
                    return '7-2',','. join(set(compress(comp_lst, lst))),title_company
                
                else:                                                                                                                   #company found at snippet level are not belonging to any company found at article
                    return '7-3','None',title_company
                #return mul_comp_title(snippet_company,title_company),title_company
            elif coref_flag==0:                                   #Snippet don't have company name
                    return '8',"coref",title_company
            else:
                return '9',title_company,title_company


In [6]:
import utils
from typing import List
from spacy.tokens import Doc, Span


In [7]:
def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
        
        final_token = document[coref[1]]
        if final_token.tag_ in ["PRP$", "POS"]:
            resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
        else:
            resolved[coref[0]] = mention_span.text + final_token.whitespace_
        for i in range(coref[0] + 1, coref[1] + 1):
            resolved[i] = ""
        
        return resolved


def original_replace_corefs(document: Doc, clusters: List[List[List[int]]]) -> str:
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
        mention_span = document[mention_start:mention_end]

        for coref in cluster[1:]:
            core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    #used to know the exact clusters which are having atleast one noun or propn
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    
    for i in range(len(noun_indices)):
        head_start, head_end = cluster[noun_indices[i]]
        head_span = doc[head_start:head_end+1]
        if(str(head_span).lower() in comp_list):
            return head_span, [head_start, head_end]
    return "null",[-1,-1]
    
def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])


def improved_replace_corefs(document, clusters):
    
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans
    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)
        if noun_indices:
            for i in range(len(noun_indices)):
                head_start, head_end = cluster[noun_indices[i]]
                head_span = document[head_start:head_end+1]
                #print(head_span)
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)
            
            if(mention!=[-1,-1]):
                #print("mention span : ",mention_span)
                for coref in cluster:
                    if coref != mention and not is_containing_other_spans(coref, all_spans):
                        #print(is_containing_other_spans(coref, all_spans))
                        core_logic_part(document, coref, resolved, mention_span)
    return "".join(resolved)          


In [8]:
predictor,nlp = utils.load_models()

/home/n12844/.local/lib/python3.8/site-packages/allennlp/tango/__init__.py:17: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  warnings.warn(
/home/n12844/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2021-11-29 09:26:45,669 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2021-11-29 09:26:46,157 - INFO - allennlp.common.file_utils - cache of https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz is up-to-date
2021-11-29 09:26:46,159 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-mode

In [9]:
def get_coref(txt):
        clusters = predictor.predict(txt)['clusters']
        tot_art = nlp(txt)
        allen1 = improved_replace_corefs(tot_art, clusters).strip()
        if(len(allen1.split("\n"))>1):
                return(allen1.split("\n")[1])
        else:
                return(txt.split("\n")[1])

        #return allen1.split("\n")[1]

In [11]:
no_tit_no_snip=0
no_tit=0
no_tit_snip=0
sin_tit_no_snip=0
yes_tit=0
sin_tit=0
mul_tit_no_snip=0
mul_tit=0
sin_tit_all_snip=0
sin_tit_any_snip=0
sin_tit_oth_snip=0
mul_tit_all_snip=0
mul_tit_any_snip=0
mul_tit_oth_snip=0
final_snippend_list=[]

for idx, row in data_aft_prep.iterrows():
    if(idx>48000):
        coref=0
        snippet_org=""
        if(str(row['text'])!='nan' and str(row['title'])!='nan'):
            value,snippet_org, title_org = tag(row['text'],row['title'],0)
        else:
            value='nan'
            snippet_org='nan'
            title_org='nan'
        if(snippet_org=="coref"):
            allen = get_coref(row['title']+"."+"\n"+row['text'])
            coref=1
            value,snippet_org, title_org = tag(allen,row['title'],1)
        temp_dict=dict()
        temp_dict['article_id'] = row['article_id']
        temp_dict['title'] = row['title']
        temp_dict['snippet'] = row['text']
        temp_dict['company_name'] = row['company_name']
        temp_dict['title_org'] = title_org
        if(coref):
            temp_dict['snippet_coref'] = allen
            temp_dict['snippet_org'] = "None"
            temp_dict['snippet_coref_org'] = snippet_org
            if(snippet_org == title_org):
                temp_dict['coref_tit_sni_same'] = 1
            else:
                temp_dict['coref_tit_sni_same'] = 0
            temp_dict['tit_sni_same'] = "None"
            temp_dict['is_coref'] = 1
        else:
            temp_dict['snippet_coref'] = "None"
            temp_dict['snippet_org'] = snippet_org
            temp_dict['snippet_coref_org'] = "None"
            if(snippet_org == title_org):
                temp_dict['tit_sni_same'] = 1
            else:
                temp_dict['tit_sni_same'] = 0
            temp_dict['coref_tit_sni_same'] = "None"
            temp_dict['is_coref'] = 0
        temp_dict['id'] = row['id']
        temp_dict['value'] = value
        if(value == '1'):
            no_tit_no_snip+=1
            no_tit+=1
        elif(value == '3'):
            no_tit_snip+=1
            no_tit+=1
        elif(value == '6'):
            sin_tit_no_snip+=1
            yes_tit+=1
            sin_tit+=1
        elif(value == '9'):
            mul_tit_no_snip+=1
            yes_tit+=1
            mul_tit+=1
        elif(value == '4-1'):
            sin_tit_all_snip+=1
            yes_tit+=1
            sin_tit+=1
        elif(value == '4-2'):
            sin_tit_any_snip+=1
            yes_tit+=1
            sin_tit+=1
        elif(value == '4-3'):
            sin_tit_oth_snip+=1
            yes_tit+=1
            sin_tit+=1
        elif(value == '7-1'):
            mul_tit_all_snip+=1
            yes_tit+=1
            mul_tit+=1
        elif(value == '7-2'):
            mul_tit_any_snip+=1
            yes_tit+=1
            mul_tit+=1
        elif(value == '7-3'):
            mul_tit_oth_snip+=1
            yes_tit+=1
            mul_tit+=1
        
        final_snippend_list.append(temp_dict)
        if(idx%6000==0):
            snippet_df = pd.DataFrame(final_snippend_list) 
            snippet_df.to_csv("emperical_result_coref_"+str(idx)+".csv")
            final_snippend_list=[]
            print("set completed")
        

set completed
set completed


In [12]:
snippet_df = pd.DataFrame(final_snippend_list) 
snippet_df.to_csv("emperical_result_coref_"+str(idx)+".csv")

In [13]:
data_0 = pd.read_csv("emperical_result_coref_0.csv")
data_6000 = pd.read_csv("emperical_result_coref_6000.csv")
data_12000 = pd.read_csv("emperical_result_coref_12000.csv")
data_18000 = pd.read_csv("emperical_result_coref_18000.csv")
data_24000 = pd.read_csv("emperical_result_coref_24000.csv")
data_30000 = pd.read_csv("emperical_result_coref_30000.csv")
data_36000 = pd.read_csv("emperical_result_coref_36000.csv")
data_42000 = pd.read_csv("emperical_result_coref_42000.csv")
data_48000 = pd.read_csv("emperical_result_coref_48000.csv")
data_54000 = pd.read_csv("emperical_result_coref_54000.csv")
data_60000 = pd.read_csv("emperical_result_coref_60000.csv")
data_64137 = pd.read_csv("emperical_result_coref_64137.csv")

In [35]:
data_aft_coref = data_0.append(data_6000)
data_aft_coref = data_aft_coref.append(data_12000)
data_aft_coref = data_aft_coref.append(data_18000)
data_aft_coref = data_aft_coref.append(data_24000)
data_aft_coref = data_aft_coref.append(data_30000)
data_aft_coref = data_aft_coref.append(data_36000)
data_aft_coref = data_aft_coref.append(data_42000)
data_aft_coref = data_aft_coref.append(data_48000)
data_aft_coref = data_aft_coref.append(data_54000)
data_aft_coref = data_aft_coref.append(data_60000)
data_aft_coref = data_aft_coref.append(data_64137)
data_aft_coref.shape

(64138, 14)

In [56]:
print(data_aft_coref[data_aft_coref['value'] == "3"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "4-1"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "4-2"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "6"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "7-1"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "7-2"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "9"]['value'].count())

3139
2800
83
6439
149
32
949


In [55]:
print(data_aft_coref[data_aft_coref['value'] == "1"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "2"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "3"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "4-1"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "4-2"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "4-3"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "5"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "6"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "7-1"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "7-2"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "7-3"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "8"]['value'].count())
print(data_aft_coref[data_aft_coref['value'] == "9"]['value'].count())
print("*********")
print(data_aft_coref[data_aft_coref['is_coref'] == 1]['is_coref'].count())
print(data_aft_coref[data_aft_coref['coref_tit_sni_same'] == "1"]['coref_tit_sni_same'].count())
print(data_aft_coref[data_aft_coref['coref_tit_sni_same'] == "0"]['coref_tit_sni_same'].count())
print(data_aft_coref[data_aft_coref['coref_tit_sni_same'] == "None"]['coref_tit_sni_same'].count())
print("*********")
print(data_aft_coref[data_aft_coref['is_coref'] == 0]['is_coref'].count())
print(data_aft_coref[data_aft_coref['tit_sni_same'] == "1"]['tit_sni_same'].count())
print(data_aft_coref[data_aft_coref['tit_sni_same'] == "0"]['tit_sni_same'].count())
print(data_aft_coref[data_aft_coref['tit_sni_same'] == "None"]['tit_sni_same'].count())

0
50260
3139
2800
83
211
0
6439
149
32
37
0
949
*********
58707
58377
329
5431
*********
5431
2205
3226
58707


In [16]:
print("Total no of sentences : ",len(data_aft_prep))
print(yes_tit)
print()
print("Percentage of sentences with company in title : "+str(round((yes_tit/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit)
print()
print("Percentage of sentences with no company in title : "+str(round((no_tit/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit)
print()
print("Percentage of sentences with single company in title : "+str(round((sin_tit/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit)
print()
print("Percentage of sentences with multiple companies in title : "+str(round((mul_tit/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit_no_snip)
print()
print("Percentage of sentences with no company in title as well as snippet : "+str(round((no_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(no_tit_snip)
print()
print("Percentage of sentences with no company in title and company in snippet : "+str(round((no_tit_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_no_snip)
print()
print("Percentage of sentences with single company in title and no company in snippet : "+str(round((sin_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_all_snip)
print()
print("Percentage of sentences with single company in title and snippet have title company : "+str(round((sin_tit_all_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_any_snip)
print()
print("Percentage of sentences with single company in title and one of the companies in snippet is title company : "+str(round((sin_tit_any_snip/len(data_aft_prep))*100,2))+"%")
print()
print(sin_tit_oth_snip)
print()
print("Percentage of sentences with single company in title and snippet company is not matching title company : "+str(round((sin_tit_oth_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_no_snip)
print()
print("Percentage of sentences with multiple companies in title and no company in snippet : "+str(round((mul_tit_no_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_all_snip)
print()
print("Percentage of sentences with multiple companies in title and snippet having one of the company in title : "+str(round((mul_tit_all_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_any_snip)
print()
print("Percentage of sentences with multiple companies in title and any one of the companies in snippet is same as one of the title company : "+str(round((mul_tit_any_snip/len(data_aft_prep))*100,2))+"%")
print()
print(mul_tit_oth_snip)
print()
print("Percentage of sentences with multiple companies in title and snippet company is matching none of the title company : "+str(round((mul_tit_oth_snip/len(data_aft_prep))*100,2))+"%")
print()

Total no of sentences :  64138
10176

Percentage of sentences with company in title : 15.87%

2572

Percentage of sentences with no company in title : 4.01%

9009

Percentage of sentences with single company in title : 14.05%

1167

Percentage of sentences with multiple companies in title : 1.82%

0

Percentage of sentences with no company in title as well as snippet : 0.0%

2572

Percentage of sentences with no company in title and company in snippet : 4.01%

6047

Percentage of sentences with single company in title and no company in snippet : 9.43%

2677

Percentage of sentences with single company in title and snippet have title company : 4.17%

80

Percentage of sentences with single company in title and one of the companies in snippet is title company : 0.12%

205

Percentage of sentences with single company in title and snippet company is not matching title company : 0.32%

949

Percentage of sentences with multiple companies in title and no company in snippet : 1.48%

149

Perc

In [ ]:
no_tit_no_snip=0
no_tit=0
no_tit_snip=0
sin_tit_no_snip=0
yes_tit=0
sin_tit=0
mul_tit_no_snip=0
mul_tit=0
sin_tit_all_snip=0
sin_tit_any_snip=0
sin_tit_oth_snip=0
mul_tit_all_snip=0
mul_tit_any_snip=0
mul_tit_oth_snip=0
final_snippend_list=[]

grouped_df = data_aft_prep.groupby(['article_id'])
for name,group in grouped_df:
    

for idx, row in tqdm(data_aft_prep.iterrows()):
    coref=0
    snippet_org=""
    if(str(row['text'])!='nan' and str(row['title'])!='nan'):
        value,snippet_org, title_org = tag(row['text'],row['title'],0)
    else:
        value='nan'
        snippet_org='nan'
        title_org='nan'
    if(snippet_org=="coref"):
        allen = get_coref(row['title']+"\n"+row['text'])
        coref=1
        value,snippet_org, title_org = tag(allen,row['title'],1)
    temp_dict=dict()
    temp_dict['article_id'] = row['article_id']
    temp_dict['title'] = row['title']
    temp_dict['snippet'] = row['text']
    temp_dict['company_name'] = row['company_name']
    temp_dict['title_org'] = title_org
    if(coref):
        temp_dict['snippet_coref'] = allen
        temp_dict['snippet_org'] = "None"
        temp_dict['snippet_coref_org'] = snippet_org
        if(snippet_org == title_org):
            temp_dict['coref_tit_sni_same'] = 1
        else:
            temp_dict['coref_tit_sni_same'] = 0
        temp_dict['tit_sni_same'] = "None"
        temp_dict['is_coref'] = 1
    else:
        temp_dict['snippet_coref'] = "None"
        temp_dict['snippet_org'] = snippet_org
        temp_dict['snippet_coref_org'] = "None"
        if(snippet_org == title_org):
            temp_dict['tit_sni_same'] = 1
        else:
            temp_dict['tit_sni_same'] = 0
        temp_dict['coref_tit_sni_same'] = "None"
        temp_dict['is_coref'] = 0
    temp_dict['id'] = row['id']
    temp_dict['value'] = value
    if(value == '1'):
        no_tit_no_snip+=1
        no_tit+=1
    elif(value == '3'):
        no_tit_snip+=1
        no_tit+=1
    elif(value == '6'):
        sin_tit_no_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '9'):
        mul_tit_no_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '4-1'):
        sin_tit_all_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '4-2'):
        sin_tit_any_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '4-3'):
        sin_tit_oth_snip+=1
        yes_tit+=1
        sin_tit+=1
    elif(value == '7-1'):
        mul_tit_all_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '7-2'):
        mul_tit_any_snip+=1
        yes_tit+=1
        mul_tit+=1
    elif(value == '7-3'):
        mul_tit_oth_snip+=1
        yes_tit+=1
        mul_tit+=1
    
    final_snippend_list.append(temp_dict)
snippet_df = pd.DataFrame(final_snippend_list) 
    

In [7]:
data_train = pd.read_csv('trainV3.csv')
data_dev = pd.read_csv('devV3.csv')
data_test = pd.read_csv('testV3.csv')
data_aug = pd.read_csv('total_aug_data.csv')
data_trainaug = pd.read_csv('trainPaugV3.csv')

In [8]:
data_train.shape, data_dev.shape, data_test.shape, data_aug.shape, data_trainaug.shape

((2083, 12), (412, 12), (462, 12), (4904, 15), (6977, 14))

In [9]:
data_train.columns, data_dev.columns, data_test.columns, data_aug.columns, data_trainaug.columns

(Index(['index', 'doccano_auto_id', 'article_id', 'prev_id', 'company_name',
        'article_title', 'text', 'ESG_label', 'difficulty', 'doccano_label',
        'sub_category', 'version'],
       dtype='object'),
 Index(['index', 'doccano_auto_id', 'article_id', 'prev_id', 'company_name',
        'article_title', 'text', 'ESG_label', 'difficulty', 'doccano_label',
        'sub_category', 'version'],
       dtype='object'),
 Index(['index', 'doccano_auto_id', 'article_id', 'prev_id', 'company_name',
        'article_title', 'text', 'ESG_label', 'difficulty', 'doccano_label',
        'sub_category', 'version'],
       dtype='object'),
 Index(['index', 'doccano_auto_id', 'article_id', 'prev_id', 'company_name',
        'article_title', 'text', 'ESG_label', 'difficulty', 'doccano_label',
        'sub_category', 'version', 'aug_type', 'ID_val', 'delete'],
       dtype='object'),
 Index(['index', 'doccano_auto_id', 'article_id', 'prev_id', 'company_name',
        'article_title', 'text', 'E

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/n12844/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
def delete_stb(df):
    
    df = df.drop_duplicates('title')
    df = df.drop_duplicates('body')
    df['article_id'] = range(0, len(df))
    df = df.sort_values('title')
    df = df.sort_values('article_id')
    return df

In [53]:
data_new2 = delete_stb(data_new)

In [54]:
data_new2.shape

(2208, 9)

In [55]:
data_new2.head(10)

,date,title,url,body,rel,hit_count,probable_esg_type,company_name,article_id
0,2021-03-30,Joseph Otis Minott: Biden must prioritize heal...,https://triblive.com/opinion/joseph-otis-minot...,Fracking machinery at a site operated by Range...,228,9,all,otis,0
1,2021-04-26,OTIS WORLDWIDE CORP (OTIS) Q1 2021 Earnings Ca...,https://www.fool.com/earnings/call-transcripts...,"Good morning, and welcome to Otis's First Quar...",162,1,all,otis,1
2,2021-02-01,OTIS WORLDWIDE CORP (OTIS) Q4 2020 Earnings Ca...,https://www.fool.com/earnings/call-transcripts...,Good morning and welcome to Otis Fourth Quarte...,136,1,all,otis,2
3,2021-06-08,﻿ Otis Unveils New Generation of Digitally Nat...,https://www.intellasia.net/%EF%BB%BFotis-unvei...,- Gen3 elevator adds built-in IoT benefits to ...,78,1,all,otis,3
4,2021-06-08,Otis Unveils New Generation of Digitally Nativ...,https://www.intellasia.net/otis-unveils-new-ge...,- Gen3 elevator adds built-in IoT benefits to ...,78,1,all,otis,4
6,2021-06-08,Otis Worldwide : Unveils New Generation of Dig...,https://www.marketscreener.com//quote/stock/OT...,"FARMINGTON, Conn., June 8, 2021 /PRNewswire/ -...",78,1,all,otis,5
7,2021-02-20,Central Maine business briefs: OTIS Federal Cr...,https://www.centralmaine.com/2021/02/20/centra...,JAY -- OTIS Federal Credit Union raised a tota...,52,4,all,otis,6
8,2021-05-24,Otis College of Art and Design's Fashion Thesi...,https://wwd.com/fashion-news/fashion-features/...,Students graduating from the fashion program a...,27,2,all,otis,7
9,2021-04-29,Today in History: Aretha Franklin's cover of O...,https://santamariatimes.com/lifestyles/today-i...,"Today is Thursday, April 29, the 119th day of ...",27,2,all,otis,8
10,2021-08-23,Why Supermodel Carré Otis Is Taking on This To...,https://www.thedailybeast.com/supermodel-carre...,"Carré Otis wants justice, but she also wants c...",2,2,all,otis,9


In [56]:
import re
def get_cleaned_text(text):
    text = re.sub("\(.*?\)", '', text)
    text = re.sub("\[.*?\]", '', text)
    text = re.sub('\*', '', text)
    text = re.sub('\s\s+', ' ', text)
    text = re.sub('\.\s\.', '.', text)
    text = re.sub(',\s,', ',', text)
    text = re.sub(';\s;', ';', text)
    text = re.sub(':\s:', ':', text)
    text = re.sub('\?\s\?', '?', text)
    text = re.sub('\!\s\!', '!', text)
    return text.strip()

def get_sents_for_doc(text):
    text = get_cleaned_text(text)
    sents = nltk.tokenize.sent_tokenize(str(text))
    len_sents = len(sents)
    final_2sents_list = []
    for i in range(1, len_sents):
        temp_text = sents[i-1] + ' ' + sents[i]
        temp_text = re.sub('"', '', temp_text)
        final_2sents_list.append(temp_text.strip())
    return final_2sents_list

def map_snippets_art(row):
    body = row['body']
    snippets = get_sents_for_doc(body)
    final_snippend_list = []
    for snip in snippets:
        temp_dict = dict()
        temp_dict['text'] = snip
        temp_dict['article_id'] = row['article_id']
        temp_dict['url'] = row['url']
        temp_dict['company_name'] = row['company_name']
        temp_dict['title'] = row['title']
        final_snippend_list.append(temp_dict)
    return final_snippend_list

In [57]:
from tqdm import tqdm

In [74]:
def get_all_snippets(art_df, start_snippet_id):
    snippets_list = []
    for idx, row in tqdm(art_df.iterrows()):
        one_snippet_list = map_snippets_art(row)
        snippets_list.extend(one_snippet_list)
    snippet_df = pd.DataFrame(snippets_list)
    snippet_df['id'] = range(start_snippet_id, len(snippet_df))
    return snippet_df

In [65]:
snippets_list = []
lst=[]
for idx, row in data_new2.iterrows():
    lst.append(idx)
    one_snippet_list = map_snippets_art(row)
    snippets_list.extend(one_snippet_list)
snippet_df = pd.DataFrame(snippets_list)
snippet_df['id'] = range(0, len(snippet_df))
    

In [66]:
id_list = list(snippet_df.id)
#id_list=lst
for i in range(1,len(id_list)):
    if(id_list[i]-id_list[i-1]!=1):
        print(id_list[i],id_list[i-1])

In [60]:
snippet_df.to_csv('snippets_bytitle_V1_NLTK.csv', index=False)

In [67]:
snippet_df.head(5)

,text,article_id,url,company_name,title,id
0,Fracking machinery at a site operated by Range...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,0
1,The U.S. Environmental Protection Agency is on...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,1
2,"Next month, Sen. Martin Heinrich, D-N.M., will...",0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,2
3,This was the first ever federal standard for m...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,3
4,We need President Biden and EPA Administrator ...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,4


In [ ]:
data_new2.to_csv('data_bytitle_V2.csv', index=False)

In [68]:
snippet_df = snippet_df.drop('id', 1)
snippet_df = snippet_df.sort_values('text')
snippet_df = snippet_df.drop_duplicates('text')
snippet_df = snippet_df.sort_values('article_id')
snippet_df['id'] = range(0, len(snippet_df))
snippet_df = snippet_df.sort_values('id')

/tmp/ipykernel_4133/764195945.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  snippet_df = snippet_df.drop('id', 1)


In [69]:
id_list = list(snippet_df.id)
#id_list=lst
for i in range(1,len(id_list)):
    if(id_list[i]-id_list[i-1]!=1):
        print(id_list[i],id_list[i-1])

In [70]:
snippet_df.head(5)

,text,article_id,url,company_name,title,id
20,If we are to meet Biden's goal of a 100% clean...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,0
30,Wind and solar must be the centerpieces of Bid...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,1
12,When methane is emitted from the oil and gas i...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,2
18,Since every step in the oil and gas production...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,3
21,This includes aggressively expanding renewable...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...,4


In [39]:
snippet_df.columns

Index(['text', 'article_id', 'url', 'company_name', 'title', 'id'], dtype='object')

In [40]:
snippet_df.to_csv('snippets_bytitle_V1_after_DupRem_NLTK.csv', index=False)

In [73]:
import re
def get_cleaned_text(text):
    text = re.sub("\(.*?\)", '', text)
    text = re.sub("\[.*?\]", '', text)
    text = re.sub('\*', '', text)
    text = re.sub('\s\s+', ' ', text)
    text = re.sub('\.\s\.', '.', text)
    text = re.sub(',\s,', ',', text)
    text = re.sub(';\s;', ';', text)
    text = re.sub(':\s:', ':', text)
    text = re.sub('\?\s\?', '?', text)
    text = re.sub('\!\s\!', '!', text)
    return text.strip()

def get_sents_for_doc(text):
    text = get_cleaned_text(text)
    #doc = nlp_spacy(text)
    #sents = list(doc.sents)
    sents = nltk.tokenize.sent_tokenize(str(text))
    len_sents = len(sents)
    final_1sents_list = []
    for i in range(0, len_sents):
        temp_text = sents[i]
        temp_text = re.sub('"', '', temp_text)
        final_1sents_list.append(temp_text.strip())
    return final_1sents_list

def map_snippets_art(row):
    body = row['body']
    snippets = get_sents_for_doc(body)
    final_snippend_list = []
    for snip in snippets:
        temp_dict = dict()
        temp_dict['text'] = snip
        temp_dict['article_id'] = row['article_id']
        temp_dict['url'] = row['url']
        temp_dict['company_name'] = row['company_name']
        temp_dict['title'] = row['title']
        final_snippend_list.append(temp_dict)
    return final_snippend_list

In [86]:
snippets_list = []
lst=[]
for idx, row in data_new2.iterrows():
    lst.append(idx)
    one_snippet_list = map_snippets_art(row)
    snippets_list.extend(one_snippet_list)
snippet_df = pd.DataFrame(snippets_list)
#snippet_df = snippet_df.drop('id', 1)
#snippet_df = snippet_df.sort_values('text')
print(snippet_df.shape)
snippet_df = snippet_df.drop_duplicates(['text','article_id'])
snipper_df = snippet_df.sort_values('article_id')
print(snippet_df.shape)
#snippet_df['id'] = range(0, len(snippet_df))
#snippet_df = snippet_df.sort_values('id')
    

(99302, 5)
(98374, 5)


In [87]:
snippet_df.head(5)

,text,article_id,url,company_name,title
0,Fracking machinery at a site operated by Range...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...
1,The U.S. Environmental Protection Agency is on...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...
2,"Next month, Sen. Martin Heinrich, D-N.M., will...",0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...
3,This was the first ever federal standard for m...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...
4,We need President Biden and EPA Administrator ...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...


In [92]:
ind_list = []
grouped_df = snippet_df.groupby(['article_id'])
for name,group in grouped_df:
    temp_dict = dict()
    text = "\n".join(list(group.text))
    temp_dict['text'] = text
    temp_dict['article_id'] = group.iloc[0].article_id
    temp_dict['url'] = group.iloc[0].url
    temp_dict['company_name'] = group.iloc[0].company_name
    temp_dict['title'] = group.iloc[0].title
    ind_list.append(temp_dict)
drop_dup_df=pd.DataFrame(ind_list)
drop_dup_df.head(10)

,text,article_id,url,company_name,title
0,Fracking machinery at a site operated by Range...,0,https://triblive.com/opinion/joseph-otis-minot...,otis,Joseph Otis Minott: Biden must prioritize heal...
1,"Good morning, and welcome to Otis's First Quar...",1,https://www.fool.com/earnings/call-transcripts...,otis,OTIS WORLDWIDE CORP (OTIS) Q1 2021 Earnings Ca...
2,Good morning and welcome to Otis Fourth Quarte...,2,https://www.fool.com/earnings/call-transcripts...,otis,OTIS WORLDWIDE CORP (OTIS) Q4 2020 Earnings Ca...
3,- Gen3 elevator adds built-in IoT benefits to ...,3,https://www.intellasia.net/%EF%BB%BFotis-unvei...,otis,﻿ Otis Unveils New Generation of Digitally Nat...
4,- Gen3 elevator adds built-in IoT benefits to ...,4,https://www.intellasia.net/otis-unveils-new-ge...,otis,Otis Unveils New Generation of Digitally Nativ...
5,"FARMINGTON, Conn., June 8, 2021 /PRNewswire/ -...",5,https://www.marketscreener.com//quote/stock/OT...,otis,Otis Worldwide : Unveils New Generation of Dig...
6,JAY -- OTIS Federal Credit Union raised a tota...,6,https://www.centralmaine.com/2021/02/20/centra...,otis,Central Maine business briefs: OTIS Federal Cr...
7,Students graduating from the fashion program a...,7,https://wwd.com/fashion-news/fashion-features/...,otis,Otis College of Art and Design's Fashion Thesi...
8,"Today is Thursday, April 29, the 119th day of ...",8,https://santamariatimes.com/lifestyles/today-i...,otis,Today in History: Aretha Franklin's cover of O...
9,"Carré Otis wants justice, but she also wants c...",9,https://www.thedailybeast.com/supermodel-carre...,otis,Why Supermodel Carré Otis Is Taking on This To...


In [93]:
drop_dup_df.to_csv('data_bytitle_V2_after_DupRem_NLTK.csv', index=False)

In [96]:
import re
def get_cleaned_text(text):
    text = re.sub("\(.*?\)", '', text)
    text = re.sub("\[.*?\]", '', text)
    text = re.sub('\*', '', text)
    text = re.sub('\s\s+', ' ', text)
    text = re.sub('\.\s\.', '.', text)
    text = re.sub(',\s,', ',', text)
    text = re.sub(';\s;', ';', text)
    text = re.sub(':\s:', ':', text)
    text = re.sub('\?\s\?', '?', text)
    text = re.sub('\!\s\!', '!', text)
    return text.strip()

def get_sents_for_doc(text):
    text = get_cleaned_text(text)
    #doc = nlp_spacy(text)
    #sents = list(doc.sents)
    sents = nltk.tokenize.sent_tokenize(str(text))
    len_sents = len(sents)
    final_2sents_list = []
    for i in range(1, len_sents):
        temp_text = sents[i-1] + ' ' + sents[i]
        temp_text = re.sub('"', '', temp_text)
        final_2sents_list.append(temp_text.strip())
    return final_2sents_list

def map_snippets_art(row):
    body = row['text']
    snippets = get_sents_for_doc(body)
    final_snippend_list = []
    for snip in snippets:
        temp_dict = dict()
        temp_dict['text'] = snip
        temp_dict['article_id'] = row['article_id']
        temp_dict['url'] = row['url']
        temp_dict['company_name'] = row['company_name']
        temp_dict['title'] = row['title']
        final_snippend_list.append(temp_dict)
    return final_snippend_list

In [97]:
snippets_list = []
lst=[]
for idx, row in drop_dup_df.iterrows():
    lst.append(idx)
    one_snippet_list = map_snippets_art(row)
    snippets_list.extend(one_snippet_list)
snippet_df = pd.DataFrame(snippets_list)
snippet_df['id'] = range(0, len(snippet_df))
    

In [98]:
snippet_df.to_csv('snippet_bytitle_V2_after_DupRem_NLTK.csv', index=False)

In [ ]:
id
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
181
182
183
184
185
186
187
188
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
